# Libraries

In [1]:
from sklearn import set_config

import pandas as pd
import polars as pl
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Configuration

In [24]:
# Global configurations for sklearn:
set_config(transform_output = "pandas")

# Global configurations for pandas:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.precision', 3)
pd.set_option('display.max_colwidth', None)

# Global configurations for polars:
pl.Config.activate_decimals(True).set_tbl_hide_column_data_types(True)
pl.Config(
    **dict(tbl_formatting = 'ASCII_FULL_CONDENSED',
            tbl_hide_column_data_types = False,
            tbl_hide_dataframe_shape = True,
            fmt_float = "mixed",
            tbl_cell_alignment = 'CENTER',
            tbl_hide_dtype_separator = True,
            tbl_cols = 100,
            tbl_rows = 50,
            fmt_str_lengths = 100,
            )
)

# Functions

In [60]:
class DataFrameProcessor:
    """ Dataframe processing class.
    """

    @staticmethod
    def convert_types(df: pl.DataFrame) -> pl.DataFrame:
        """ Converts columns' data types for memory.

        Argument:
        df (polars dataframe): The DataFrame to be processed.
        """
        for column in df.columns:
            if column == "target":
                df = df.with_columns(pl.col(column).cast(pl.Int8))
            elif column in ["case_id", "WEEK_NUM", "num_group1", "num_group2"]:
                df = df.with_columns(pl.col(column).cast(pl.Int32))
            elif column == "date_decision":
                df = df.with_columns(pl.col(column).cast(pl.Date))
            elif column[-1] in ("P", "A"):
                df = df.with_columns(pl.col(column).cast(pl.Float64))
            elif column[-1] == "M":
                df = df.with_columns(pl.col(column).cast(pl.String))
            elif column[-1] == "D":
                df = df.with_columns(pl.col(column).cast(pl.Date))            

        return df
    
    @staticmethod
    def date_processor(df: pl.DataFrame) -> pl.DataFrame:
        """ Processes the date columns.

        Argument:
        df (polars dataframe): The DataFrame to be processed.
        """
        for column in df.columns:
            if column[-1] == "D":
                df = df.with_columns(pl.col(col) - pl.col("date_decision"))
                df = df.with_columns(pl.col(col).dt.total_days())
                df = df.with_columns(pl.col(col).cast(pl.Float32))

            if column == "date_decision":
                df = df.with_columns(
                    pl.col(column).dt.year().alias("year_decision"),
                    pl.col(column).dt.month().alias("month_decision"),
                    pl.col(column).dt.day().alias("day_decision"),
                    pl.col(column).dt.week().alias("week_decision"),
                    pl.col(column).dt.weekday().alias("weekday_decision"),
                    pl.col(column).dt.quarter().alias("quarter_decision")
                )

        df = df.drop(["date_decision", "WEEK_NUM", "MONTH"])
        
        return df
    
    @staticmethod
    def delete_nulls_column(df: pl.DataFrame) -> pl.DataFrame:
        """ Deletes columns with more than 80% of null values.

        Argument:
        df (polars dataframe): The DataFrame to be processed.
        """
        for column in df.columns:
            if column not in ["case_id", "target"]:
                null_percentage = df[column].is_null().sum() / df.shape[0]

                if null_percentage > 0.80:
                    df = df.drop(column)
        
        return df
    
    @staticmethod
    def drop_duplicates(df: pl.DataFrame) -> pl.DataFrame:
        """ Drops duplicates from the DataFrame.

        Argument:
        df (polars dataframe): The DataFrame to be processed.
        """
        df = df.unique(keep="first")
        
        return df
    
    @staticmethod
    def drop_columns_with_too_many_categories(df: pl.DataFrame) -> pl.DataFrame:
        """ Drops columns with more than 100 categories or just 1.

        Argument:
        df (polars dataframe): The DataFrame to be processed.
        """
        for column in df.columns:
            if (column not in ["target", "case_id"]) & (df[column].dtype == pl.String):
                categories_count = df[column].n_unique()

                if (categories_count == 1) | (categories_count > 200):
                    df = df.drop(column)

        return df

In [72]:
class Aggregator:
    """Dataframe aggreagating class."""

    @staticmethod
    def max_agg(df: pl.DataFrame) -> pl.Expr:
        """Aggregates the DataFrame by the maximum value.

        Argument:
        df (polars dataframe): The DataFrame to be aggregated.
        """
        columns = [column for column in df.columns if column[-1] in ("P", "A")]

        expr_max = [pl.max(column).alias(f"max_{column}") for column in columns]

        return expr_max

    @staticmethod
    def min_agg(df: pl.DataFrame) -> pl.Expr:
        """Aggregates the DataFrame by the minimum value.

        Argument:
        df (polars dataframe): The DataFrame to be aggregated.
        """
        columns = [column for column in df.columns if column[-1] in ("P", "A")]

        expr_min = [pl.min(column).alias(f"min_{column}") for column in columns]

        return expr_min

    @staticmethod
    def date_agg(df: pl.DataFrame) -> pl.Expr:
        """Aggregates the DataFrame by the date columns.

        Argument:
        df (polars dataframe): The DataFrame to be aggregated.
        """
        columns = [column for column in df.columns if column[-1] == "D"]

        expr_date = [pl.max(column).alias(f"max_{column}") for column in columns]

        return expr_date

    @staticmethod
    def string_agg(df: pl.DataFrame) -> pl.Expr:
        """Aggregates the DataFrame by the string columns.

        Argument:
        df (polars dataframe): The DataFrame to be aggregated.
        """
        columns = [column for column in df.columns if column[-1] == "M"]

        expr_string = [pl.max(column).alias(f"max_{column}") for column in columns]

        return expr_string

    @staticmethod
    def others_agg(df: pl.DataFrame) -> pl.Expr:
        """Aggregates the DataFrame by the other columns.

        Argument:
        df (polars dataframe): The DataFrame to be aggregated.
        """
        columns = [column for column in df.columns if column in ("T", "L")]

        expr_others = [pl.max(column).alias(f"max_{column}") for column in columns]

        return expr_others

    @staticmethod
    def count_agg(df: pl.DataFrame) -> pl.Expr:
        """Aggregates the DataFrame by the count of rows.

        Argument:
        df (polars dataframe): The DataFrame to be aggregated.
        """
        columns = [column for column in df.columns if "num_group" in column]

        expr_max = [pl.max(column).alias(f"max_{column}") for column in columns]

        return expr_max

    @staticmethod
    def agg_expr(df: pl.DataFrame) -> pl.Expr:
        expr_all = (
            Aggregator.max_agg(df)
            + Aggregator.min_agg(df)
            + Aggregator.date_agg(df)
            + Aggregator.string_agg(df)
            + Aggregator.others_agg(df)
            + Aggregator.count_agg(df)
        )
        return expr_all

In [3]:
def read_file(path: str, depth: int) -> pl.DataFrame:
    df = pl.read_parquet(path)
    df = df.pipe(DataFrameProcessor.convert_types)

    if depth in (1, 2):
        df = df.group_by('case_id').agg(Aggregator.agg_expr(df))

    return df


def read_files(paths: list, depth: int) -> pl.DataFrame:
    dfs = [read_file(path, depth) for path in paths]
    df = pl.concat(dfs)

    return df

# Read Data

In [21]:
%%time
# Test pandas reading time
test_pandas_df = pd.read_parquet('data/train/train_base.parquet')
test_pandas_df.head()

CPU times: user 54.3 ms, sys: 71.3 ms, total: 126 ms
Wall time: 150 ms


,case_id,date_decision,MONTH,WEEK_NUM,target
0,0,2019-01-03,201901,0,0
1,1,2019-01-03,201901,0,0
2,2,2019-01-04,201901,0,0
3,3,2019-01-03,201901,0,0
4,4,2019-01-04,201901,0,1


In [53]:
%%time
df_train = pl.read_parquet('data/train/train_base.parquet')
display(df_train.head())

print(df_train.shape)

case_id,date_decision,MONTH,WEEK_NUM,target
i64,str,i64,i64,i64
0,"""2019-01-03""",201901,0,0
1,"""2019-01-03""",201901,0,0
2,"""2019-01-04""",201901,0,0
3,"""2019-01-03""",201901,0,0
4,"""2019-01-04""",201901,0,1


(1526659, 5)
CPU times: user 70.3 ms, sys: 27.4 ms, total: 97.7 ms
Wall time: 41.3 ms


> Comment: Polars read data faster than pandas, since there are many files, it might be a good idea for me to mitigate to Polars.

In [58]:
df_train = df_train.pipe(DataFrameProcessor.convert_types)
df_train = df_train.pipe(DataFrameProcessor.date_processor)

display(df_train.head())
print(df_train.shape)
df_train = df_train.pipe(DataFrameProcessor.delete_nulls_column)
df_train = df_train.pipe(DataFrameProcessor.drop_duplicates)

print(df_train.shape)

case_id,target,year_decision,month_decision,day_decision,week_decision,weekday_decision,quarter_decision
i32,i8,i32,i8,i8,i8,i8,i8
0,0,2019,1,3,1,4,1
1,0,2019,1,3,1,4,1
2,0,2019,1,4,1,5,1
3,0,2019,1,3,1,4,1
4,1,2019,1,4,1,5,1


(1526659, 8)
(1526659, 8)


In [61]:
train_debit = pl.read_parquet('data/train/train_debitcard_1.parquet')
train_debit.head()

case_id,last180dayaveragebalance_704A,last180dayturnover_1134A,last30dayturnover_651A,num_group1,openingdate_857D
i64,f64,f64,f64,i64,str
225,null,null,null,0,"""2016-08-16"""
331,null,null,null,0,"""2015-03-19"""
358,null,null,null,0,"""2014-09-02"""
390,null,null,null,0,"""2014-07-23"""
390,null,null,null,1,"""2015-10-01"""


In [67]:
train_person = pl.read_parquet('data/train/train_person_2.parquet')
train_person.head()

case_id,addres_district_368M,addres_role_871L,addres_zip_823M,conts_role_79M,empls_economicalst_849M,empls_employedfrom_796D,empls_employer_name_740M,num_group1,num_group2,relatedpersons_role_762T
i64,str,str,str,str,str,str,str,i64,i64,str
5,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""a55475b1""",null,"""a55475b1""",0,0,null
6,"""P55_110_32""","""CONTACT""","""P10_68_40""","""P38_92_157""","""P164_110_33""",null,"""a55475b1""",0,0,null
6,"""P55_110_32""","""PERMANENT""","""P10_68_40""","""a55475b1""","""a55475b1""",null,"""a55475b1""",0,1,null
6,"""P204_92_178""","""CONTACT""","""P65_136_169""","""P38_92_157""","""P164_110_33""",null,"""a55475b1""",1,0,"""OTHER_RELATIVE"""
6,"""P191_109_75""","""CONTACT""","""P10_68_40""","""P7_147_157""","""a55475b1""",null,"""a55475b1""",1,1,"""OTHER_RELATIVE"""


In [71]:
train_person.group_by("case_id").agg(pl.col("addres_district_368M").max())

case_id,addres_district_368M
i64,str
251566,"""a55475b1"""
1930547,"""a55475b1"""
2586625,"""a55475b1"""
1881392,"""a55475b1"""
2615871,"""a55475b1"""
2657955,"""a55475b1"""
1774201,"""a55475b1"""
2670498,"""a55475b1"""
1389479,"""a55475b1"""


# Reference

* [Home Credit Baseline](https://www.kaggle.com/code/greysky/home-credit-baseline)